# Install

In [27]:
!pip install sentencepiece

# Evn

In [3]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [4]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [5]:
!nvidia-smi

Tue Jan 26 13:21:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# data dir
data_dir = '/content/drive/MyDrive'
os.listdir(data_dir)

['Colab Notebooks',
 '01-02-simple-project-과제.ipynb',
 'ko_32000.model',
 'ko_32000.vocab',
 '02-01-encoding-강의.ipynb',
 '.ipynb_checkpoints',
 'kowiki',
 '02-02-tokenizer-강의.ipynb',
 '02-02-tokenizer-과제.ipynb']

In [9]:
# kowiki dir
kowiki_dir = os.path.join(data_dir, 'kowiki')
if not os.path.exists(kowiki_dir):
    os.makedirs(kowiki_dir)
os.listdir(kowiki_dir)

['kowiki.txt.zip', 'my_corpus.txt']

# Corpus

In [10]:
text = """위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다. 2002년 10월에 새로운 위키 소프트웨어를 쓰면서 한글 처리 문제가 풀리기 시작했지만, 가장 많은 사람이 쓰는 인터넷 익스플로러에서는 인코딩 문제가 여전했다. 이런 이유로 초기에는 도움말을 옮기거나 쓰는 일에 어려움을 겪었다. 이런 어려움이 있었는데도 위키백과 통계로는, 2002년 10월에서 2003년 7월까지 열 달 사이에 글이 13개에서 159개로 늘었고 2003년 7월과 8월 사이에는 한 달 만에 159개에서 348개로 늘어났다. 2003년 9월부터는 인터넷 익스플로러의 인코딩 문제가 사라졌으며, 대한민국 언론에서도 몇 차례 위키백과를 소개하면서 참여자가 점증하리라고 예측했다. 참고로 한국어 위키백과의 최초 문서는 2002년 10월 12일에 등재된 지미 카터 문서이다.
2005년 6월 5일 양자장론 문서 등재를 기점으로 총 등재 문서 수가 1만 개를 돌파하였고 이어 동해 11월에 제1회 정보트러스트 어워드 인터넷 문화 일반 분야에 선정되었다. 2007년 8월 9일에는 한겨레21에서 한국어 위키백과와 위키백과 오프라인 첫 모임을 취재한 기사를 표지 이야기로 다루었다.
2008년 광우병 촛불 시위 때 생긴 신조어인 명박산성이 한국어 위키백과에 등재되고 이 문서의 존치 여부를 두고 갑론을박의 과정이 화제가 되고 각종 매체에도 보도가 되었다. 시위대의 난입과 충돌을 방지하기 위해 거리에 설치되었던 컨테이너 박스를 이명박 정부의 불통으로 풍자를 하기 위해 사용된 이 신조어는 중립성을 지켰는지와 백과사전에 올라올 만한 문서인지가 쟁점이 되었는데 일시적으로 사용된 신조어일 뿐이라는 주장과 이미 여러 매체에서 사용되어 지속성이 보장되었다는 주장 등 논쟁이 벌어졌고 다음 아고라 등지에서 이 항목을 존치하는 방안을 지지하는 의견을 남기기 위해 여러 사람이 새로 가입하는 등 혼란이 빚어졌다. 11월 4일에는 다음커뮤니케이션에서 글로벌 세계 대백과사전을 기증받았으며, 2009년 3월에는 서울특별시로부터 콘텐츠를 기증받았다. 2009년 6월 4일에는 액세스권 등재를 기점으로 10만 개 문서 수를 돌파했다.
2011년 4월 16일에는 대한민국에서의 위키미디어 프로젝트를 지원하는 모임을 결성할 것을 추진하는 논의가 이뤄졌고 이후 창립준비위원회 결성을 거쳐 2014년 10월 19일 창립총회를 개최하였으며, 최종적으로 2015년 11월 4일 사단법인 한국 위키미디어 협회가 결성되어 활동 중에 있다. 2019년 미국 위키미디어재단으로부터 한국 지역 지부(챕터)로 승인을 받았다.
2012년 5월 19일에는 보비 탬블링 등재를 기점으로 총 20만 개 문서가 등재되었고 2015년 1월 5일, Rojo -Tierra- 문서 등재를 기점으로 총 30만 개 문서가 등재되었다. 2017년 10월 21일에는 충청남도 동물위생시험소 문서 등재로 40만 개의 문서까지 등재되었다."""

# Google sentencepiece를 이용해 vocab 생성
- https://github.com/google/sentencepiece

## sentencepe 학습

In [11]:
os.listdir(kowiki_dir)

['kowiki.txt.zip', 'my_corpus.txt']

In [12]:
shutil.copy(os.path.join(kowiki_dir, 'my_corpus.txt'), './')
os.listdir('./')

['.config', 'drive', 'my_corpus.txt', 'sample_data']

In [13]:
def train_sentencepiece(corpus, prefix, vocab_size=32000):
    """
    sentencepiece를 이용해 vocab 학습
    :param corpus: 학습할 말뭉치
    :param prefix: 저장할 vocab 이름
    :param vocab_size: vocab 개수
    """
    spm.SentencePieceTrainer.train(
        f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +  # 7은 특수문자 개수
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" +  # pad token 및 id 지정
        " --unk_id=1 --unk_piece=[UNK]" +  # unknown token 및 id 지정
        " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence token 및 id 지정
        " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence token 및 id 지정
        " --user_defined_symbols=[SEP],[CLS],[MASK]")  # 기타 추가 토큰 SEP: 4, CLS: 5, MASK: 6

In [15]:
# vocab 생성
train_sentencepiece(f"my_corpus.txt", f"my_vocab", vocab_size=425)

In [16]:
os.listdir(".")

['.config',
 'my_vocab.vocab',
 'my_vocab.model',
 'drive',
 'my_corpus.txt',
 'sample_data']

## sentencepe 확인

In [17]:
# load vocab
vocab = spm.SentencePieceProcessor()
vocab.load(f"my_vocab.model")

True

In [18]:
# vocab 출력
print(f"len: {len(vocab)}")
for id in range(16):
    print(f"{id:2d}: {vocab.id_to_piece(id)}")

len: 432
 0: [PAD]
 1: [UNK]
 2: [BOS]
 3: [EOS]
 4: [SEP]
 5: [CLS]
 6: [MASK]
 7: ▁
 8: 는
 9: .
10: 을
11: 고
12: 이
13: 에
14: 다
15: "


In [19]:
# text를 tokenize 함
# sentence to pieces
pieces = vocab.encode_as_pieces(text)
print(pieces[0:100])

['▁', '위', '키', '백', '과', '의', '▁최', '상', '위', '▁', '도', '메', '인', '이', '▁', '.', 'c', 'o', 'm', '이', '던', '▁시', '절', '▁', 'k', 'o', '.', 'wi', 'k', 'ip', 'e', 'di', 'a', '.', 'c', 'o', 'm', '에', '▁', '구판', '▁', '미디', '어', '위', '키', '가', '▁', '깔렸', '으', '나', '▁한', '글', '▁', '처', '리', '에', '▁', '문', '제', '가', '▁있', '어', '▁', '글', '을', '▁', '올', '릴', '▁수', '도', '▁', '없', '는', '▁이', '름뿐', '인', '▁', '곳', '이', '었', '다', '.', '▁2', '0', '0', '2', '년', '▁1', '0', '월', '에', '▁', '새', '로', '운', '▁', '위', '키', '▁', '소']


In [20]:
vocab.encode_as_pieces("나는 오늘 수원에 놀러 갔다.")

['▁',
 '나',
 '는',
 '▁',
 '오',
 '늘',
 '▁수',
 '원',
 '에',
 '▁',
 '놀',
 '러',
 '▁',
 '갔',
 '다',
 '.']

In [21]:
vocab.encode_as_pieces("나는 오늘 배가많이고파서밥을많이먹은후 놀러 갔다.")

['▁',
 '나',
 '는',
 '▁',
 '오',
 '늘',
 '▁',
 '배',
 '가',
 '많',
 '이',
 '고',
 '파',
 '서',
 '밥',
 '을',
 '많',
 '이',
 '먹',
 '은',
 '후',
 '▁',
 '놀',
 '러',
 '▁',
 '갔',
 '다',
 '.']

In [22]:
# tokenize된 값을 string 으로 복원
# pieces to sentence
vocab.decode_pieces(pieces)

'위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다. 2002년 10월에 새로운 위키 소프트웨어를 쓰면서 한글 처리 문제가 풀리기 시작했지만, 가장 많은 사람이 쓰는 인터넷 익스플로러에서는 인코딩 문제가 여전했다. 이런 이유로 초기에는 도움말을 옮기거나 쓰는 일에 어려움을 겪었다. 이런 어려움이 있었는데도 위키백과 통계로는, 2002년 10월에서 2003년 7월까지 열 달 사이에 글이 13개에서 159개로 늘었고 2003년 7월과 8월 사이에는 한 달 만에 159개에서 348개로 늘어났다. 2003년 9월부터는 인터넷 익스플로러의 인코딩 문제가 사라졌으며, 대한민국 언론에서도 몇 차례 위키백과를 소개하면서 참여자가 점증하리라고 예측했다. 참고로 한국어 위키백과의 최초 문서는 2002년 10월 12일에 등재된 지미 카터 문서이다. 2005년 6월 5일 양자장론 문서 등재를 기점으로 총 등재 문서 수가 1만 개를 돌파하였고 이어 동해 11월에 제1회 정보트러스트 어워드 인터넷 문화 일반 분야에 선정되었다. 2007년 8월 9일에는 한겨레21에서 한국어 위키백과와 위키백과 오프라인 첫 모임을 취재한 기사를 표지 이야기로 다루었다. 2008년 광우병 촛불 시위 때 생긴 신조어인 명박산성이 한국어 위키백과에 등재되고 이 문서의 존치 여부를 두고 갑론을박의 과정이 화제가 되고 각종 매체에도 보도가 되었다. 시위대의 난입과 충돌을 방지하기 위해 거리에 설치되었던 컨테이너 박스를 이명박 정부의 불통으로 풍자를 하기 위해 사용된 이 신조어는 중립성을 지켰는지와 백과사전에 올라올 만한 문서인지가 쟁점이 되었는데 일시적으로 사용된 신조어일 뿐이라는 주장과 이미 여러 매체에서 사용되어 지속성이 보장되었다는 주장 등 논쟁이 벌어졌고 다음 아고라 등지에서 이 항목을 존치하는 방안을 지지하는 의견을 남기기 위해 여러 사람이 새로 가입하는 등 혼란이 빚어졌다. 11월 4일

In [23]:
# tokenize된 값을 id로 변경
# piece to id
piece_ids = []
for piece in pieces:
    piece_ids.append(vocab.piece_to_id(piece))
piece_ids[0:100]

[7,
 171,
 1,
 400,
 63,
 48,
 236,
 65,
 171,
 7,
 58,
 1,
 35,
 12,
 7,
 9,
 370,
 373,
 1,
 12,
 190,
 72,
 1,
 7,
 213,
 373,
 9,
 1,
 213,
 1,
 371,
 1,
 215,
 9,
 370,
 373,
 1,
 13,
 7,
 1,
 7,
 1,
 66,
 171,
 1,
 18,
 7,
 1,
 50,
 26,
 158,
 1,
 7,
 426,
 170,
 13,
 7,
 401,
 86,
 18,
 160,
 66,
 7,
 1,
 10,
 7,
 1,
 305,
 54,
 58,
 7,
 1,
 8,
 42,
 1,
 35,
 7,
 1,
 12,
 1,
 14,
 9,
 140,
 128,
 128,
 251,
 1,
 101,
 128,
 1,
 13,
 7,
 1,
 20,
 1,
 7,
 171,
 1,
 7,
 317]

In [24]:
# text를 id로 tokenize 함
# sentence to ids
ids = vocab.encode_as_ids(text)
print(ids)

[7, 171, 1, 400, 63, 48, 236, 65, 171, 7, 58, 1, 35, 12, 7, 9, 370, 373, 1, 12, 190, 72, 1, 7, 213, 373, 9, 1, 213, 1, 371, 1, 215, 9, 370, 373, 1, 13, 7, 1, 7, 1, 66, 171, 1, 18, 7, 1, 50, 26, 158, 1, 7, 426, 170, 13, 7, 401, 86, 18, 160, 66, 7, 1, 10, 7, 1, 305, 54, 58, 7, 1, 8, 42, 1, 35, 7, 1, 12, 1, 14, 9, 140, 128, 128, 251, 1, 101, 128, 1, 13, 7, 1, 20, 1, 7, 171, 1, 7, 317, 1, 66, 23, 7, 1, 21, 17, 158, 1, 7, 426, 170, 7, 401, 86, 18, 7, 1, 170, 244, 72, 123, 425, 228, 27, 7, 18, 260, 7, 365, 29, 142, 1, 12, 7, 1, 8, 7, 35, 67, 1, 7, 1, 200, 1, 20, 119, 44, 8, 7, 35, 88, 1, 7, 401, 86, 18, 7, 122, 157, 24, 9, 42, 1, 42, 397, 20, 7, 1, 244, 13, 8, 7, 58, 1, 412, 10, 7, 1, 244, 428, 26, 7, 1, 8, 7, 47, 13, 7, 66, 106, 1, 10, 7, 287, 1, 14, 9, 42, 1, 7, 66, 106, 1, 12, 160, 1, 8, 298, 58, 7, 171, 1, 400, 63, 7, 349, 75, 20, 8, 27, 140, 128, 128, 251, 1, 101, 128, 1, 44, 140, 128, 128, 211, 1, 7, 175, 1, 293, 22, 181, 7, 165, 142, 12, 13, 7, 1, 12, 101, 211, 147, 44, 101, 255, 79, 

In [25]:
# tokenize된 id 값을 string 으로 복원
# id to sentence
vocab.decode_ids(ids)

'위 ⁇ 백과의 최상위 도 ⁇ 인이 .co ⁇ 이던 시 ⁇  ko. ⁇ k ⁇ e ⁇ a.co ⁇ 에  ⁇   ⁇ 어위 ⁇ 가  ⁇ 으나 한 ⁇  처리에 문제가 있어  ⁇ 을  ⁇ 릴 수도  ⁇ 는 이 ⁇ 인  ⁇ 이 ⁇ 다. 2002 ⁇  10 ⁇ 에  ⁇ 로 ⁇  위 ⁇  소 ⁇ 어를  ⁇ 면서 한 ⁇  처리 문제가  ⁇ 리기 시작했지만, 가장 많은 사 ⁇ 이  ⁇ 는 인터 ⁇   ⁇ 스 ⁇ 로러에서는 인코 ⁇  문제가 여전했다. 이 ⁇  이유로  ⁇ 기에는 도 ⁇ 말을  ⁇ 기거나  ⁇ 는 일에 어려 ⁇ 을 겪 ⁇ 다. 이 ⁇  어려 ⁇ 이 있 ⁇ 는데도 위 ⁇ 백과 통계로는, 2002 ⁇  10 ⁇ 에서 2003 ⁇  7 ⁇ 까지 열 달 사이에  ⁇ 이 13개에서 159개로  ⁇ 고 2003 ⁇  7 ⁇ 과 8 ⁇  사이에는 한 달 만에 159개에서 348개로  ⁇ 어났다. 2003 ⁇  9 ⁇ 부터는 인터 ⁇   ⁇ 스 ⁇ 로러의 인코 ⁇  문제가 사라 ⁇ 으며, 대한민 ⁇   ⁇ 에서도  ⁇  차례 위 ⁇ 백과를 소개하면서 참여자가  ⁇ 증하리라고 예 ⁇ 했다. 참고로 한 ⁇ 어 위 ⁇ 백과의 최 ⁇  문서는 2002 ⁇  10 ⁇  12일에 등재된 지 ⁇  카터 문서이다. 2005 ⁇  6 ⁇  5일 양자장 ⁇  문서 등재를 기 ⁇ 으로 총 등재 문서 수가 1만 개를  ⁇ 파하 ⁇ 고 이어 동해 11 ⁇ 에 제1회 정보 ⁇ 러스 ⁇  어 ⁇ 드 인터 ⁇  문화 일반 분야에 선정되 ⁇ 다. 2007 ⁇  8 ⁇  9일에는 한 ⁇ 21에서 한 ⁇ 어 위 ⁇ 백과와 위 ⁇ 백과 오 ⁇ 라인  ⁇  모임을  ⁇ 재한 기사를 표지 이야기로 다 ⁇ 다. 2008 ⁇  광우 ⁇   ⁇  시위 때 생 ⁇  신조어인 명 ⁇ 산성이 한 ⁇ 어 위 ⁇ 백과에 등재되고 이 문서의 존치 여부를 두고  ⁇ 을 ⁇ 의 과정이 화제가 되고 각종  ⁇ 체에도 보도가 되 ⁇ 다. 시위대의 난 ⁇ 과 충 ⁇ 을 방지하기 위해 거리에 설치되 ⁇ 던  ⁇ 테이 ⁇   ⁇ 스를 이명 ⁇

In [26]:
# id 값을 token으로 변경
# id to piece
id_pieces = []
for id in ids:
    id_pieces.append(vocab.id_to_piece(id))
id_pieces[0:100]

['▁',
 '위',
 '[UNK]',
 '백',
 '과',
 '의',
 '▁최',
 '상',
 '위',
 '▁',
 '도',
 '[UNK]',
 '인',
 '이',
 '▁',
 '.',
 'c',
 'o',
 '[UNK]',
 '이',
 '던',
 '▁시',
 '[UNK]',
 '▁',
 'k',
 'o',
 '.',
 '[UNK]',
 'k',
 '[UNK]',
 'e',
 '[UNK]',
 'a',
 '.',
 'c',
 'o',
 '[UNK]',
 '에',
 '▁',
 '[UNK]',
 '▁',
 '[UNK]',
 '어',
 '위',
 '[UNK]',
 '가',
 '▁',
 '[UNK]',
 '으',
 '나',
 '▁한',
 '[UNK]',
 '▁',
 '처',
 '리',
 '에',
 '▁',
 '문',
 '제',
 '가',
 '▁있',
 '어',
 '▁',
 '[UNK]',
 '을',
 '▁',
 '[UNK]',
 '릴',
 '▁수',
 '도',
 '▁',
 '[UNK]',
 '는',
 '▁이',
 '[UNK]',
 '인',
 '▁',
 '[UNK]',
 '이',
 '[UNK]',
 '다',
 '.',
 '▁2',
 '0',
 '0',
 '2',
 '[UNK]',
 '▁1',
 '0',
 '[UNK]',
 '에',
 '▁',
 '[UNK]',
 '로',
 '[UNK]',
 '▁',
 '위',
 '[UNK]',
 '▁',
 '소']